In [1]:
import subprocess
import os
import time
import csv

TIMEOUT = 120
REPETITIONS = 5
ENUMERATION_LIMIT = 1000

In [2]:
# Runs trapmvn with either deadlock or siphon encoding on the given model, 
# and reports the average runtime, number of failed experiments, plus the number
# of detected fixed-points.
def bench_trapmvn(model, fixed_point_method = "deadlock"):
    cmd_trapmvn = ["python3", "-m", "trapmvn", "-c", "fix", "-s", "unitary", "-m", str(ENUMERATION_LIMIT), "-fm", fixed_point_method, model]
    total_time = 0
    fails = 0
    result_count = 0    
    for _i in range(REPETITIONS):
        try:
            start = time.perf_counter()
            result = subprocess.run(cmd_trapmvn, capture_output=True, timeout=TIMEOUT)
            total_time += time.perf_counter() - start
            result_count = len(result.stdout.splitlines()) - 1
            if result.returncode != 0:
                fails += 1
        except subprocess.TimeoutExpired:
            total_time += TIMEOUT
            fails += 1
    return (float(total_time) / float(REPETITIONS), fails, result_count)

In [3]:
def bench_trappist(model, fixed_point_method = "1"):
    model_bnet = model.replace(".sbml", ".bnet")
    cmd_trappist = ["python3", "-m", "trappist", "-s", "asp", "-c", "fix", "-m", str(ENUMERATION_LIMIT), "-fm", fixed_point_method, model_bnet]
    total_time = 0
    fails = 0
    result_count = 0    
    for _i in range(REPETITIONS):
        try:
            start = time.perf_counter()
            # First, create a bnet model file.
            result = subprocess.call(['java', '-jar', 'bioLQM.jar', model, model_bnet], stdout=subprocess.DEVNULL)
            assert result == 0
            # Then run trappist on this bnet file.
            result = subprocess.run(cmd_trappist, capture_output=True, timeout=TIMEOUT)
            total_time += time.perf_counter() - start            
            result_count = len(result.stdout.splitlines()) - 1            
            if result.returncode != 0:
                fails += 1
        except subprocess.TimeoutExpired:
            total_time += TIMEOUT
            fails += 1
    return (float(total_time) / float(REPETITIONS), fails, result_count) 

In [4]:
def process_output_an_asp(output: str):
    lines = output.split("\\n")
    n_fixed_points = 0    
    for line in lines:
        if "Models       :" in line:
            tmp = (line.split(":"))[1]            
            if tmp.endswith("+"):
                tmp = tmp[:-1]                
            n_fixed_points = int(tmp)
            break            
    return n_fixed_points

# For some reason, the "fails" counter does not work for AN-ASP,
# but the results seem to be correct nevertheless.
def bench_an_asp(model):
    model_an = model.replace(".sbml", ".an")
    model_lp = model.replace(".sbml", ".lp")
    total_time = 0
    fails = 0
    result_count = 0    
    for _i in range(REPETITIONS):
        try:
            start = time.perf_counter()
            # First, convert .sbml to .an
            result = subprocess.call(['java', '-jar', 'bioLQM.jar', model, model_an], stdout=subprocess.DEVNULL)
            assert result == 0
            # Then use AN-ASP to produce a logic program
            result = subprocess.call(['python3', 'AN2asp.py', model_an, model_lp], stdout=subprocess.DEVNULL)
            assert result == 0
            output = subprocess.run(['clingo', str(ENUMERATION_LIMIT), 'fixed-points.lp', '-q', model_lp], capture_output=True, timeout=TIMEOUT)
            total_time += time.perf_counter() - start            
            result_count = process_output_an_asp(str(output.stdout))
            if output.returncode != 0:
                fails += 1
        except subprocess.TimeoutExpired:
            total_time += TIMEOUT
            fails += 1
    return (float(total_time) / float(REPETITIONS), fails, result_count) 

In [5]:
# Here, non-monotonic models report as fail with 0 results.
def bench_mpbn(model):
    model_bnet = model.replace(".sbml", ".bnet")
    total_time = 0
    fails = 0
    result_count = 0
    for _i in range(REPETITIONS):
        try:
            start = time.perf_counter()
            # First, convert .sbml to .bnet
            result = subprocess.call(['java', '-jar', 'bioLQM.jar', model, model_bnet], stdout=subprocess.DEVNULL)
            assert result == 0
            # Then use mpbn
            output = subprocess.run(['python3', 'mpbn_helper.py', model_bnet, "fix", str(ENUMERATION_LIMIT)], capture_output=True, timeout=TIMEOUT)
            total_time += time.perf_counter() - start            
            result_count = len(output.stdout.splitlines())  
            if output.returncode != 0:
                fails += 1
        except subprocess.TimeoutExpired:
            total_time += TIMEOUT
            fails += 1
    return (float(total_time) / float(REPETITIONS), fails, result_count) 

In [6]:
with open('fixed-point-benchmark.tsv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t')
    
    header = ["Model"]
    header += ["trapmvn (siphon)", "trapmvn (deadlock)"]
    header += ["trappist (1)", "trappist (2)"]
    header += ["mpbn"]
    header += ["an-asp"]
    writer.writerow(header)
    print(header)
    
    for model in sorted(os.listdir("models")):
        if not model.endswith(".sbml"):
            continue
        
        row = [model]
        model = f"models/{model}"
        # Fixed points are shared across semantics.
        trapmvn_siphon = bench_trapmvn(model, fixed_point_method="siphon")
        trapmvn_deadlock = bench_trapmvn(model, fixed_point_method="deadlock")        
        row += [str(trapmvn_siphon), str(trapmvn_deadlock)]
        trappist_1 = bench_trappist(model, fixed_point_method="1")
        trappist_2 = bench_trappist(model, fixed_point_method="2")
        row += [str(trappist_1), str(trappist_2)]
        mpbn = bench_mpbn(model)
        row += [str(mpbn)]
        an_asp = bench_an_asp(model)
        row += [str(an_asp)]
        print(row)
        
        writer.writerow(row)
        csvfile.flush()

['Model', 'trapmvn (siphon)', 'trapmvn (deadlock)', 'trappist (1)', 'trappist (2)', 'mpbn', 'an-asp']
['146_BUDDING-YEAST-FAURE-2009.sbml', '(1.301216052286327, 0, 0)', '(0.37864248994737865, 0, 0)', '(3.674640227295458, 0, 0)', '(2.0153969280421733, 0, 0)', '(120.0, 5, 0)', '(1.4370259856805205, 5, 0)']
['148_AGS-cell-fate-decision.sbml', '(0.10808874946087599, 0, 1)', '(0.10771687142550945, 0, 1)', '(0.934755103290081, 0, 1)', '(0.9259763387963176, 0, 1)', '(1.3061180064454674, 0, 1)', '(0.7903871713206172, 5, 1)']
['151_TCR-REDOX-METABOLISM.sbml', '(0.2720512034371495, 0, 9)', '(0.23659965563565494, 0, 9)', '(1.1412842124700546, 0, 9)', '(1.10178816113621, 0, 9)', '(15.895261129364371, 0, 9)', '(0.8949557084590196, 5, 9)']
['155_CONTROL-OF-TH1-TH2-TH17-TREG-DIFFERENTATION.sbml', '(0.34897585324943065, 0, 1000)', '(0.32718475721776485, 0, 1000)', '(0.9747584197670222, 0, 1000)', '(0.9740484213456512, 0, 1000)', '(1.2568450467661023, 5, 0)', '(0.7866102021187544, 5, 1000)']
['157_CONT